In [1]:
import pandas as pd
import re
from geopy.geocoders import Nominatim
from time import sleep
import numpy as np

In [2]:
# --- 1. LEER EL ARCHIVO CSV CON LA CODIFICACIÓN CORRECTA ---
# La causa principal del error está aquí. Añadimos encoding='utf-8'.
try:
    df = pd.read_csv('datos_casinos_salas.csv', encoding='utf-8')
    print("✅ Archivo 'datos_casinos_salas.csv' leído correctamente con UTF-8.")
except UnicodeDecodeError:
    print("⚠️ UTF-8 falló. Intentando con 'latin-1'...")
    df = pd.read_csv('datos_casinos_salas.csv', encoding='latin-1')
    print("✅ Archivo leído con 'latin-1'.")
except FileNotFoundError:
    print("❌ Error: No se encontró el archivo 'datos_casinos_salas.csv'.")
    # Si el archivo no existe, creamos uno de ejemplo para que el script no falle.
    data = {'Dirección': ['AV. ALFREDO BENAVIDES N° 509', 'JR. DE LA UNIÓN NRO. 456'],
            'Distrito': ['MIRAFLORES', 'LIMA'],
            'Provincia': ['LIMA', 'LIMA'],
            'Departamento': ['LIMA', 'LIMA']}
    df = pd.DataFrame(data)

✅ Archivo 'datos_casinos_salas.csv' leído correctamente con UTF-8.


In [5]:
# --- 2. LIMPIAR Y PREPARAR LAS DIRECCIONES ---
def clean_address(row):
    """Combina y limpia los campos de dirección para un formato estándar."""
    # Convertimos todos los campos a string para evitar errores
    address = str(row['Dirección'])
    district = str(row['Distrito'])
    province = str(row['Provincia'])
    department = str(row['Departamento'])

    # La limpieza se mantiene igual, pero ahora operará sobre texto bien decodificado
    address = re.sub(r'No\.?|N°|Nº|Nro\.?', '#', address).strip()
    
    # Formato completo de la dirección para la geocodificación
    full_address = f"{address}, {district}, {province}, {department}, Perú"
    return full_address

df['Direccion_Completa'] = df.apply(clean_address, axis=1)

In [6]:
# --- 3. GEOCODIFICAR DIRECCIONES ---
geolocator = Nominatim(user_agent="geocodificador_corregido")

def get_coordinates_with_status(address):
    """Geocodifica una dirección y devuelve latitud, longitud y estado."""
    try:
        sleep(1) # Pausa para respetar los límites del servicio de geocodificación
        location = geolocator.geocode(address, timeout=10)
        
        if location:
            print(f"📍 Encontrado: {address}")
            return pd.Series([location.latitude, location.longitude, 'Encontrado'])
        else:
            print(f"❌ No encontrado: {address}")
            return pd.Series([None, None, 'No Encontrado'])
    except Exception as e:
        print(f"⚠️ Error en '{address}': {str(e)}")
        return pd.Series([None, None, f'Error: {str(e)}'])

print("\nIniciando proceso de geocodificación...")
# Aplicamos la función para obtener Latitud, Longitud y Estado
df[['Latitud', 'Longitud', 'Estado']] = df['Direccion_Completa'].apply(get_coordinates_with_status)
print("\n✅ Proceso de geocodificación terminado.")


Iniciando proceso de geocodificación...
📍 Encontrado: ALFREDO BENAVIDES # 430, MIRAFLORES, LIMA, LIMA, Perú
📍 Encontrado: Calle SAN MARTIN # 275 - 275, TUMBES, TUMBES, TUMBES, Perú
📍 Encontrado: Av. JOSE PARDO # 422 - 422, MIRAFLORES, LIMA, LIMA, Perú
📍 Encontrado: Av. MALECON DE LA RESERVA # 615, MIRAFLORES, LIMA, LIMA, Perú
📍 Encontrado: Av. LA MARINA # 1725 - 1729, SAN MIGUEL, LIMA, LIMA, Perú
📍 Encontrado: Av. ARENALES 1814-A # 1814, LINCE, LIMA, LIMA, Perú
📍 Encontrado: Av. CAYMA # 701, CAYMA, AREQUIPA, AREQUIPA, Perú
📍 Encontrado: Calle MERCADERES # 303, AREQUIPA, AREQUIPA, AREQUIPA, Perú
❌ No encontrado: Otro PORTAL DE FLORES # 104 - 106 - 110 - 116 - 118 - 120, AREQUIPA, AREQUIPA, AREQUIPA, Perú
📍 Encontrado: Calle SAN CAMILO # 102-A - 104, AREQUIPA, AREQUIPA, AREQUIPA, Perú
📍 Encontrado: Jr. SAN ROMAN # 301, JULIACA, SAN ROMÁN, PUNO, Perú
❌ No encontrado: Jr. LORETO # 228, TAMBOPATA, TAMBOPATA, MADRE DE DIOS, Perú
❌ No encontrado: Av. 28 DE JULIO Y AV. LARCO 895 # 531, MIRAFL

In [7]:
# --- 4. CONTAR Y MOSTRAR RESUMEN ---
encontrados = df[df['Estado'] == 'Encontrado'].shape[0]
no_encontrados = df[df['Estado'] != 'Encontrado'].shape[0]

print("\n--- Resumen de Resultados ---")
print(f"Direcciones encontradas: {encontrados}")
print(f"Direcciones no encontradas o con error: {no_encontrados}")
print("---------------------------\n")


--- Resumen de Resultados ---
Direcciones encontradas: 340
Direcciones no encontradas o con error: 352
---------------------------



In [ ]:
'''# --- 5. GUARDAR EL RESULTADO FINAL EN CSV ---
# Seleccionamos las columnas que queremos en el archivo final
columnas_finales = ['Direccion_Completa', 'Estado', 'Latitud', 'Longitud']
df_final = df[columnas_finales]

# Definimos el nombre del archivo de salida
nombre_archivo_salida = 'V2_resultados_geocodificados_corregido.csv'

# Guardamos en CSV, asegurando usar UTF-8 para preservar los caracteres especiales
df_final.to_csv(nombre_archivo_salida, index=False, encoding='utf-8-sig')

print(f"🎉 ¡Proceso completo! Resultados guardados sin errores en '{nombre_archivo_salida}'")

# Mostramos una vista previa del resultado final
print("\nVista previa de los datos guardados:")
print(df_final.head())'''